# Business Objective
##### I will analyze health insurance data to predict future insurance claim amount and identify factors leading to higher costs. The data consists of 1,338 people with 7 attributes that describe health and demographic information such as  age, gender, BMI, and smoking status.

# Installing required modules

In [4]:
! pip install numpy
! pip install pandas
! pip install seaborn
! pip install matplotlib
! pip install plotly

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


You should consider upgrading via the 'C:\Users\VAClemor\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

You should consider upgrading via the 'C:\Users\VAClemor\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.



Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     -------------------------------------- 292.8/292.8 KB 3.0 MB/s eta 0:00:00


You should consider upgrading via the 'C:\Users\VAClemor\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


You should consider upgrading via the 'C:\Users\VAClemor\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

You should consider upgrading via the 'C:\Users\VAClemor\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


# Data Overview

In [7]:
! pip install sklearn

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 7.3/7.3 MB 18.1 MB/s eta 0:00:00
     ---------------------------------------- 307.0/307.0 KB ? eta 0:00:00
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1309 sha256=9f3f0a974f011ef09a2d780a356013561aac1fe1d97edcd337e75693ff0fc644
  Stored in directory: C:\Users\VAClemor\AppData\Local\Temp\pip-ephem-wheel-cache-nr6hkwtm\wheels\9b\13\01\6f3a7fd641f90e1f6c8c7cded057f3394f451f340371c68f3d
Successfully built sklearn


You should consider upgrading via the 'C:\Users\VAClemor\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [73]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.offline import plot, iplot, init_notebook_mode
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import r2_score, mean_squared_error

### Loading Data

In [74]:
datafile = pd.read_csv("./insurance.csv")

In [75]:
datafile

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
...,...,...,...,...,...,...,...
1333,50,male,30.970,3,no,northwest,10600.54830
1334,18,female,31.920,0,no,northeast,2205.98080
1335,18,female,36.850,0,no,southeast,1629.83350
1336,21,female,25.800,0,no,southwest,2007.94500


### Validating data integrity 

In [76]:
print("There are {:,} records and {} columns in this data set".format(datafile.shape[0], datafile.shape[1]))
print("There are {} missing values in the data as shown below".format(datafile.isna().sum().sum()))
print("{}".format(datafile.isna().sum()))
datafile['sex'] = datafile['sex'].str.capitalize()
datafile['smoker'] = datafile['smoker'].apply(lambda x: 'Smoker' if x=='yes' else 'Non-Smoker')
datafile['region'] = datafile['region'].str.capitalize()

There are 1,338 records and 7 columns in this data set
There are 0 missing values in the data as shown below
age         0
sex         0
bmi         0
children    0
smoker      0
region      0
charges     0
dtype: int64


### Summary Statistics of Data

In [77]:
datafile.describe().T

,count,mean,std,min,25%,50%,75%,max
age,1338.0,39.207025,14.049960,18.0000,27.00000,39.000,51.000000,64.00000
bmi,1338.0,30.663397,6.098187,15.9600,26.29625,30.400,34.693750,53.13000
children,1338.0,1.094918,1.205493,0.0000,0.00000,1.000,2.000000,5.00000
charges,1338.0,13270.422265,12110.011237,1121.8739,4740.28715,9382.033,16639.912515,63770.42801


In [78]:
### Data Categorical Statistics

In [79]:
datafile.select_dtypes(include=['object']).describe()

,sex,smoker,region
count,1338,1338,1338
unique,2,2,4
top,Male,Non-Smoker,Southeast
freq,676,1064,364


### Finding Average Claim Amount

In [80]:
cost = datafile.select_dtypes(include=['object']).columns.tolist()
for i in cost:
    total = datafile[i].value_counts()
    claim_amount = datafile.groupby(i)["charges"].mean()
    display(pd.DataFrame({"Number of individuals" :total, "Claim Amount" :claim_amount.map('${:,.2f}'.format)})\
        .sort_values("Number of individuals", ascending=False)\
        .style.set_caption("Variable by {}".format(i)))

,Number of individuals,Claim Amount
Male,676,"$13,956.75"
Female,662,"$12,569.58"


,Number of individuals,Claim Amount
Non-Smoker,1064,"$8,434.27"
Smoker,274,"$32,050.23"


,Number of individuals,Claim Amount
Southeast,364,"$14,735.41"
Northwest,325,"$12,417.58"
Southwest,325,"$12,346.94"
Northeast,324,"$13,406.38"
